In [1]:
# Basic packages
import pandas as pd 
import numpy as np
import re
import collections

# Packages for data preparation
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

max_words = 10000  # Parameter indicating the number of words we'll put in the dictionary

df = pd.read_csv('input/airline/Tweets.csv')
df = df.reindex(np.random.permutation(df.index))  
df = df[['text', 'airline_sentiment']]

import nltk
nltk.download('stopwords')

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
def remove_mentions(input_text):
        return re.sub(r'@\w+', '', input_text)
       
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)

X_train, X_test, y_train, y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.1, random_state=37)
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

tk = Tokenizer(num_words=max_words,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(X_train)

X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)

from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(X_train_seq, maxlen=26)
labels = np.asarray(y_train_le)

training_samples = 10248  # We will be training on 10248 samples
validation_samples = 2928

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = to_categorical(labels)

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(max_words, 32, input_length=26))
model.add(LSTM(32,
              dropout=0.2,
              recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.summary()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liyuantan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 26, 32)            320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 328,419
Trainable params: 328,419
Non-trainable params: 0
_________________________________________________________________


In [2]:
%%time
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=6,
                    batch_size=64)
model.save_weights('lstm_airline_test.h5')

Epoch 1/6
10248/10248 [==============================] - 6s 539us/step - loss: 0.7901 - acc: 0.6657
Epoch 2/6
10248/10248 [==============================] - 5s 458us/step - loss: 0.6040 - acc: 0.7498
Epoch 3/6
10248/10248 [==============================] - 5s 469us/step - loss: 0.5113 - acc: 0.7977
Epoch 4/6
10248/10248 [==============================] - 5s 494us/step - loss: 0.4503 - acc: 0.8278
Epoch 5/6
10248/10248 [==============================] - 5s 486us/step - loss: 0.4036 - acc: 0.8470
Epoch 6/6
10248/10248 [==============================] - 5s 469us/step - loss: 0.3675 - acc: 0.8592
CPU times: user 58.5 s, sys: 14.6 s, total: 1min 13s
Wall time: 30.9 s


Collect the training time on the training set.

In [3]:
x_test = pad_sequences(X_test_seq, maxlen=26)
y_test = np.asarray(y_test_le)
y_test = to_categorical(y_test)

In [4]:
%%time
model.evaluate(x_test, y_test)

1464/1464 [==============================] - 0s 260us/step
CPU times: user 573 ms, sys: 119 ms, total: 691 ms
Wall time: 383 ms


[0.5313918437462687, 0.8005464480874317]

Collect the test accuracy on the test set.